In [251]:
import requests
from google.cloud import bigquery
from datetime import datetime, timedelta
import json

In [ ]:
def get_news(company_list, api_key: str, sortBy='relevance', language='en', from_date=None, to_date=datetime.datetime.now().strftime('%Y-%m-%d')) -> pd.DataFrame:
    """
    This function returns the news articles for a given company from the News API.
    """
    df_list = []
    for company in company_list:

        api_search = f'https://newsapi.org/v2/everything?q={company}&from={
            from_date}&to={to_date}&sortBy={sortBy}&language={language}&apiKey={api_key}'

        try:
            response = requests.get(api_search).json()
            if response.get('status') == 'ok':

                # Extract specific information
                articles = response.get('articles', [])
                extracted_info = []
                for article in articles:
                    source_name = article.get('source', {}).get('name')
                    title = article.get('title')
                    description = article.get('description')
                    published_at = article.get('publishedAt')
                    extracted_info.append({
                        'company': company,
                        'source_name': source_name,
                        'title': title,
                        'description': description,
                        'published_at': published_at
                    })

            else:
                print('Error in the API response')
                extracted_info = []

        except requests.exceptions.RequestException as e:
            print(e)
            print(response.status_code)
        df = pd.DataFrame(extracted_info)
        df_list.append(df)

    df = pd.concat(df_list)

    return df

In [256]:
def get_data(company: str, api_key: str, from_date=(datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d'), to_date=datetime.now().strftime('%Y-%m-%d'), sort_by='relevance', language='en'):

    url = f'https://newsapi.org/v2/everything?q={company}&from={from_date}&to={
        to_date}&sortBy={sort_by}&language={language}&apiKey={api_key}'
    response = requests.get(url=url)
    if response.status_code == 200:
        return response.json()
    else:
        print('fel!')

In [257]:
data_raw = get_data(
    company='Apple', api_key='d3a5fafb5fc4415fb7f4f33a2a399250')

In [245]:
def write(data, company: str, table='table_1', project_id='tomastestproject-433206', dataset='testdb_1') -> None:

    # Initiera BigQuery-klienten
    client = bigquery.Client.from_service_account_json(
        '/Users/tomasrydenstam/Downloads/tomastestproject-433206-adc5bc090976.json')

    # Definiera din dataset och tabell
    table_id = f"{project_id}.{dataset}.{table}"
    print(table_id)

    rows_to_insert = []

    for row in data['articles']:
        row['source'] = row['source']['name']
        row['company'] = company
        row['fetch_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        rows_to_insert.append(row)
    print(rows_to_insert)

    errors = client.insert_rows_json(table_id, rows_to_insert)
    print(errors)

In [259]:
data_raw

{'status': 'ok',
 'totalResults': 283,
 'articles': [{'source': {'id': None, 'name': 'MacRumors'},
   'author': 'Tim Hardwick',
   'title': 'Kuo: Folding All-Screen MacBook Delayed Until Late 2027 At the Earliest',
   'description': "Apple's rumored foldable MacBook won't arrive until the end of 2027 or 2028 at the earliest because of technical challenges, according to the latest prediction by Apple analyst Ming-Chi Kuo.\n\n\n\nConcept by Astropad\n\nPrevious reports from display industry analy…",
   'url': 'https://www.macrumors.com/2024/08/22/kuo-foldable-macbook-delayed-2027/',
   'urlToImage': 'https://images.macrumors.com/t/zKktJKfU2gMwxRwH3vDHqQb2HYs=/1600x/article-new/2024/05/Foldable-Screen-MacBook-Concept-Astropad.jpg',
   'publishedAt': '2024-08-22T10:50:25Z',
   'content': "Apple's rumored foldable MacBook won't arrive until the end of 2027 or 2028 at the earliest because of technical challenges, according to the latest prediction by Apple analyst Ming-Chi Kuo.\r\nPreviou… [

In [ ]:
def main(company):
    data = get_data(company)
    write(data, company)

In [189]:
test = json.dumps(data_raw['articles'])

In [202]:
rows_to_insert = []
for x in data_raw['articles']:
    x['source'] = x['source']['name']
    rows_to_insert.append(x)

In [221]:
data_raw

{'status': 'ok',
 'totalResults': 1784,
 'articles': [{'source': {'id': None, 'name': 'Yahoo Entertainment'},
   'author': 'Lawrence Bonk',
   'title': 'Apple event rumor roundup: What to expect at the iPhone 16 keynote',
   'description': 'The biggest day of the tech world’s fall calendar is nearly here: Apple’s iPhone 16 event will take place sometime in September. The company typically does it on a Tuesday during the first half of the month, so September 10 is a safe bet. No matter when it ha…',
   'url': 'https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_7de77318-ab44-441e-a0d7-b47c8388fcdd',
   'urlToImage': None,
   'publishedAt': '2024-08-20T19:23:47Z',
   'content': "If you click 'Accept all', we and our partners, including 237 who are part of the IAB Transparency &amp; Consent Framework, will also store and/or access information on a device (in other words, use … [+678 chars]"},
  {'source': {'id': 'the-verge', 'name': 'The Verge'},
   'author': 'Antonio G. Di

In [219]:
write(data_raw)

tomastestproject-433206.testdb_1.test_3


TypeError: json_rows argument should be a sequence of dicts